# Import Libraries

In [1]:
import sys
sys.path.append("/home/felix/PycharmProjects/Quantum-Challenge/")
import pandas as pd
import xarray as xr
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import traceback
import math

In [2]:
import networkx as nx
import itertools

from qiskit.algorithms import AmplificationProblem
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import Grover
from qiskit.circuit.library import GroverOperator
from qiskit.extensions import Initialize

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

# import basic plot tools
from qiskit.visualization import plot_histogram

from itertools import product 
from qiskit.quantum_info import Statevector

# Keywords

Classical approach: selection of the trajectory with the shortest Euclidean distance to the target in order to minimize the distance traveled, i.e. save fuel and time

try to find argument on how many planes can fit into single voxel with

# TODO

- Two seperate Quantum Models: Grovers search algorithm to find a climate optimized trajectory and FVQE to de-conflict computed trajectories. 


- Simulate plane trajec with timestamps in mp4 (animated plot)
- 0.75 Euro/kg for fuel and 25 Euro/min for crew costs, based on work within the REACT4C project
- Argue for discretization i.e. continuus optimal control problem cannot be solved easitly with current qubit number, therefore disc is used to simplify and then interpolated to regenerate the traj in the end. THe interpolation assumes smooth trajectories with neglegible delta climate impact compared to the disc traj.
- De-conflicitng

# Define Boundaries

x: Longitude in deg

y: Latitude in deg

z: Flight Level

t: Timestampe in DateTime format

In [106]:
max_x = 30 
min_x = -30 

max_y = 60  
min_y = 34  

spacing_xy = 2

max_z = 400
min_z = 100
spacing_z = 20

In [107]:
x_arr = np.arange(min_x, max_x + spacing_xy, spacing_xy)
y_arr  = np.arange(min_y, max_y + spacing_xy, spacing_xy)
z_arr   = np.arange(min_z, max_z + spacing_z, spacing_z)

box_3d = np.zeros((x_arr.shape[0], y_arr.shape[0], z_arr.shape[0]))

# Load flight Data 

In [108]:
flight_df = pd.read_csv("../data/flights.csv", sep=";")

# Load velocity and fuel consumption data

In [110]:
fuel_df = pd.read_csv("../data/bada_data.csv", sep=";")
cruise_df_temp = pd.read_csv("../data/bada_data.csv", sep=";").iloc[:,[0,1,2]]
climb_df = pd.read_csv("../data/bada_data.csv", sep=";").iloc[:,[0,3,4,5]]
descent_df = pd.read_csv("../data/bada_data.csv", sep=";").iloc[:,[0,6,7,8]]

In [111]:
cruise_df = cruise_df_temp.drop(index=[0,1,2,3,4,5,6,7,8]).reset_index() # remove FL below 100
cruise_df.drop(cruise_df[cruise_df.FL<200].index, inplace=True)
cruise_df = cruise_df.reset_index()
cruise_df.drop(columns=["level_0","index"],inplace=True)

In [112]:
fl = cruise_df["FL"].to_numpy()
tas = cruise_df["TAS [kts]"].to_numpy()
fuel = cruise_df["fuel [kg/min]"].to_numpy()

# Load climate data

In [113]:
def xyz_to_tupel(x, y, z):
    if math.isnan(z):
        return (float("nan"),float("nan"),float("nan"))
    else:
        return (  int((x+30)/2), int((y-34)/2), int((z-100)/20))

In [114]:
convert_dict = {
    600: (100, 120, 140), 
    550: (160, float("nan"), float("nan")), 
    500: (180, float("nan"), float("nan")), 
    450: (200, 220, float("nan")), 
    400: (240, float("nan"), float("nan")), 
    350: (260, 280, float("nan")), 
    300: (300, 320, float("nan")),
    250: (340, float("nan"), float("nan")), 
    225: (360, float("nan"), float("nan")), 
    200: (380, 400, float("nan"))} 

In [115]:
ds = xr.open_dataset("../data/aCCF_0623_p_spec.nc")
climate_df_temp = ds.to_dataframe()
climate_df = climate_df_temp.reset_index()
climate_df.drop(columns=["LEVEL11_24_bnds","bnds","SG_NOX","SG_DCONT","SG_H2O","E022"], inplace=True)
climate_df.drop(climate_df[climate_df.LONGITUDE<-30].index, inplace=True)
climate_df.reset_index(inplace=True)

hpa_old = climate_df["LEVEL11_24"].to_numpy()
fl_mapped = []#np.zeros(len(hpa_old))
rem_index = []

for i in range(len(hpa_old)):
    if hpa_old[i] in convert_dict:
        fl_mapped.append( convert_dict[hpa_old[i]])
    else:
        fl_mapped.append(float("nan")  )
        rem_index.append(i)

climate_df["FL"] = fl_mapped
climate_df.drop(rem_index, inplace=True)
climate_df.reset_index(inplace=True)
climate_df.drop(columns=["level_0","index"],inplace=True)

    

temp_x = climate_df["LONGITUDE"].to_numpy()
temp_y = climate_df["LATITUDE"].to_numpy()
temp_z = climate_df["FL"].to_numpy()

tupel_arr = []
for x,y,temp_z_arr in zip(temp_x, temp_y, temp_z):
    temp_tupel = ()
    for z in temp_z_arr:
        temp_tupel = (xyz_to_tupel(x,y,z),) + temp_tupel 
    tupel_arr += [temp_tupel[::-1]]
climate_df["tupel"] = tupel_arr 
climate_df.drop_duplicates(inplace=True)
climate_df.reset_index(inplace=True)
climate_df.drop(columns=["index"],inplace=True)
climate_df

,LONGITUDE,LATITUDE,LEVEL11_24,TIME,MERGED,FL,tupel
0,-30.0,34.0,200,2018-06-23 06:00:00,0.088344,"(380, 400, nan)","((0, 0, 14), (0, 0, 15), (nan, nan, nan))"
1,-30.0,34.0,200,2018-06-23 12:00:00,0.088190,"(380, 400, nan)","((0, 0, 14), (0, 0, 15), (nan, nan, nan))"
2,-30.0,34.0,200,2018-06-23 18:00:00,0.088036,"(380, 400, nan)","((0, 0, 14), (0, 0, 15), (nan, nan, nan))"
3,-30.0,34.0,225,2018-06-23 06:00:00,0.084318,"(360, nan, nan)","((0, 0, 13), (nan, nan, nan), (nan, nan, nan))"
4,-30.0,34.0,225,2018-06-23 12:00:00,0.084189,"(360, nan, nan)","((0, 0, 13), (nan, nan, nan), (nan, nan, nan))"
...,...,...,...,...,...,...,...
13015,30.0,60.0,550,2018-06-23 12:00:00,0.112334,"(160, nan, nan)","((30, 13, 3), (nan, nan, nan), (nan, nan, nan))"
13016,30.0,60.0,550,2018-06-23 18:00:00,0.114098,"(160, nan, nan)","((30, 13, 3), (nan, nan, nan), (nan, nan, nan))"
13017,30.0,60.0,600,2018-06-23 06:00:00,0.116333,"(100, 120, 140)","((30, 13, 0), (30, 13, 1), (30, 13, 2))"
13018,30.0,60.0,600,2018-06-23 12:00:00,0.118673,"(100, 120, 140)","((30, 13, 0), (30, 13, 1), (30, 13, 2))"


# Define functions to relate measured data to parametrized voxels

In [116]:
def x_to_km(deg):
    "return km"
    return deg*85

def y_to_km(deg):
    "return km"
    return deg*111

In [117]:
def find_x(x_hat):
    inx = (np.abs(x_arr - x_hat)).argmin()
    return x_arr[inx]

def find_y(y_hat):
    inx = (np.abs(y_arr - y_hat)).argmin()
    return y_arr[inx]

def find_z(z_hat):
    inx = (np.abs(z_arr - z_hat)).argmin()
    return z_arr[inx]

In [118]:
def find_z_arr(z_hat):
    
    nearest_val = z_hat
    difference = z_hat
    
    for cnt, arr in enumerate(convert_dict.values()):
        for test_fl in arr:
            if np.abs(test_fl - z_hat) < difference:
                nearest_val = arr
                difference = np.abs(test_fl - z_hat)
    
    return nearest_val

In [119]:
def find_fuel(df, FL_val):
    """"returns TAS and fuel consumption in kg/min """
    inx = (np.abs(df["FL"].to_numpy() - FL_val )).argmin()
   
    cl_inx = df.index[inx]
    
    return df.iloc[cl_inx].values[1],float(df.iloc[cl_inx].values[2].replace(",","."))

# find_fuel(cruise_df,250)

In [120]:
def find_fuel_climb(df, FL_val):
    """"returns TAS and fuel consumption in kg/min """
    inx = (np.abs(df["FL"].to_numpy() - FL_val )).argmin()
   
    cl_inx = df.index[inx]
    
    return df.iloc[cl_inx].values[1], df.iloc[cl_inx].values[2], float(df.iloc[cl_inx].values[3].replace(",","."))


In [224]:
#The trajectory for each airplane is defined as a list of points, altitudes, and time stamps
def consumed_fuel(trajectory):
    c_fuel = 0 
    
    c_fuel_dict = {0:[trajectory[0]["x"],trajectory[0]["y"],trajectory[0]["z"],0]}
    for cnt in range(len(trajectory)-1):
        temp_fuel = 0 
        
        
        delta_x = np.abs(trajectory[cnt+1]["x"] - trajectory[cnt]["x"] )
        delta_y = np.abs(trajectory[cnt+1]["y"] - trajectory[cnt]["y"] )
        z_0 = trajectory[cnt]["z"] 
        z_1 = trajectory[cnt+1]["z"] 
        
        if z_1 > z_0: 
            
            _, roc_0, c_r_c_0 = find_fuel_climb(climb_df, z_0)
            _, roc_1, c_r_c_1 = find_fuel_climb(climb_df, z_1)
            delta_t_c_0 = 1000/roc_0 # in minutes 
            delta_t_c_1 = 1000/roc_1 # in minutes 
            
            
            temp_fuel += c_r_c_0 * delta_t_c_0
            temp_fuel += c_r_c_1 * delta_t_c_1
            
            consumption_rate_cruise = find_fuel(cruise_df,z_1)[-1]
            delta_t = np.abs(trajectory[cnt+1]["t"] - trajectory[cnt]["t"] ).astype(float)/60 - delta_t_c_0 - delta_t_c_1
            temp_fuel += consumption_rate_cruise * delta_t
            
        elif z_1 < z_0:
            _, roc_0, c_r_c_0 = find_fuel_climb(descent_df, z_0)
            _, roc_1, c_r_c_1 = find_fuel_climb(descent_df, z_1)
            delta_t_c_0 = 1000/roc_0 # in minutes 
            delta_t_c_1 = 1000/roc_1 # in minutes 
            
            temp_fuel += c_r_c_0 * delta_t_c_0
            temp_fuel += c_r_c_1 * delta_t_c_1
            
            consumption_rate_cruise = find_fuel(cruise_df,z_1)[-1]
            delta_t = np.abs(trajectory[cnt+1]["t"] - trajectory[cnt]["t"] ).astype(float)/60 - delta_t_c_0 - delta_t_c_1
            temp_fuel += consumption_rate_cruise * delta_t
            
        else:
        
            consumption_rate = find_fuel(cruise_df,z_0)[-1] # in kg/min
            delta_t = np.abs(trajectory[cnt+1]["t"] - trajectory[cnt]["t"] ).astype(float)/(60)

            temp_fuel += consumption_rate * delta_t 

        c_fuel_dict[cnt+1] = [trajectory[cnt+1]["x"],trajectory[cnt+1]["y"],z_0,temp_fuel]
        c_fuel += temp_fuel
        
    return c_fuel, c_fuel_dict


In [122]:
def find_climate(df, x, y, z, t):
    
    scaled_x = find_x(x)
    scaled_y = find_y(y)
    scaled_z = find_z_arr(z)

    temp_series = df.loc[(df["LONGITUDE"] == scaled_x) & (df["LATITUDE"] == scaled_y) & (df["FL"] == scaled_z), "TIME"].dt.hour

    inx = (np.abs(temp_series.to_numpy() - ( (t-t.astype('datetime64[D]'))/3600).astype(int)   )).argmin()
    r_inx = temp_series.index[inx]
    ret = df.loc[(df["LONGITUDE"] == scaled_x) & (df["LATITUDE"] == scaled_y) & (df["FL"] == scaled_z) & (df["TIME"] == np.datetime64('2018-06-23') + np.timedelta64(temp_series.loc[r_inx],'h') ), "MERGED"]
    
    return ret 

# find_climate(climate_df, x=30, y=56, z=400, t=np.datetime64("2018-06-23 06:00:00"))

# Define navigation functions

In [123]:
def rad_change_direction(v,angle):
    rad = angle*np.pi/180
    return v**2/(np.tan(rad))*0.0000269
rad_change_direction(400,25)
# normalized to km

9.229957785873141

In [124]:
170*222*0.06096/(0.3048*9.26*279)

2.9215727257948396

I.e. max 3 planes in a voxel at a given time 

# Define Cost function and constraints

In [125]:
# todo: overlap at different times 

In [126]:
final_x_arr = flight_df.end_longitudinal.to_numpy()
final_y_arr = flight_df.end_latitudinal.to_numpy()
def constraint_final_xy(trajec_arr):
    x_error = 0
    y_error = 0 
    for cnt, traj in enumerate(trajec_arr):
        if traj[-1]["x"] != final_x_arr[cnt]:
            x_error += 1
        elif traj[-1]["y"] != final_y_arr[cnt]:
            y_error += 1
    return x_error + y_error
# constraint_final_xy(classical_trajectory_arr)

In [127]:
def constraint_path(trajec_arr):
    path_error = 0
    
    for cnt, traj in enumerate(trajec_arr):
        for step in range(len(traj)-1):
            delta_x = np.abs(traj[step+1]["x"] - traj[step]["x"])
            delta_y = np.abs(traj[step+1]["y"] - traj[step]["y"])
            delta_z = np.abs(traj[step+1]["z"] - traj[step]["z"])
            
            if   delta_x == 2 and delta_y == 0 and delta_z == 0:
                1
            elif delta_x == 0 and delta_y == 2 and delta_z == 0:
                1
            elif delta_x == 2 and delta_y == 2 and delta_z == 0:
                1
            elif delta_x == 2 and delta_y == 0 and delta_z == 20:
                1
            elif delta_x == 0 and delta_y == 2 and delta_z == 20:
                1
            elif delta_x == 2 and delta_y == 2 and delta_z == 20:
                1
            else:
                path_error += 1
                
    return path_error

# constraint_path(classical_trajectory_arr)

In [128]:
# constraining parameters
con_vertical = 0.3048 # km
con_horizont = 9.26 # km
con_inclination = 25 # deg only above this
p = 100 

In [129]:
def C(trajectory):
    
    c_fuel = consumed_fuel(trajectory)[0] # kg 
    
    #delta_CO2 = 6.94 * 10**(-12) # K/kg fuel
    
    effect_sum = np.sum([find_climate(climate_df, x=point["x"], y=point["y"], z=point["z"], t=point["t"]) for point in trajectory])
    
    delta_c = effect_sum*10**(-12) # K/kg fuel
    
    return (delta_c)*c_fuel

# Optimize single flight

In [130]:
max_x_tuple = 30 
min_x_tuple = 0 

max_y_tuple = 13  
min_y_tuple = 0  

max_z_tuple = 15
min_z_tuple = 0

In [131]:
def get_3d_graph():
    G = nx.grid_graph(dim=(z_arr.shape[0],y_arr.shape[0],x_arr.shape[0]))

    for e in G.edges():

        if e[0][-1] != e[1][-1] and e[0][0] == e[1][0] and  e[0][1] == e[1][1]:
            G.remove_edge(*e)
            


    add_edge_arr = []
    for n in G.nodes(): 
        if   n[0] + 1 < max_x_tuple and n[-1] + 1 < max_z_tuple:
             e = (n,(n[0] + 1,n[1],n[-1] + 1))
             add_edge_arr.append(e)

        elif n[0] + 1 < max_x_tuple and n[-1] - 1 < min_z_tuple:
             e = (n,(n[0] + 1,n[1],n[-1] - 1))
             add_edge_arr.append(e)

        elif n[0] - 1 < min_x_tuple and n[-1] + 1 < max_z_tuple:
             e = (n,(n[0] - 1,n[1],n[-1] + 1))
             add_edge_arr.append(e)

        elif n[0] - 1 < min_x_tuple and n[-1] - 1 < min_z_tuple:
             e = (n,(n[0] - 1,n[1],n[-1] - 1))
             add_edge_arr.append(e)

        elif n[1] + 1 < max_y_tuple and n[-1] + 1 < max_z_tuple:
             e = (n,(n[0],n[1] + 1,n[-1] + 1))
             add_edge_arr.append(e)

        elif n[1] + 1 < max_y_tuple and n[-1] - 1 < min_z_tuple:
             e = (n,(n[0],n[1] + 1,n[-1] - 1))
             add_edge_arr.append(e)

        elif n[1] - 1 < min_y_tuple and n[-1] + 1 < max_z_tuple:
             e = (n,(n[0],n[1] - 1,n[-1] + 1))
             add_edge_arr.append(e)

        elif n[1] - 1 < min_y_tuple and n[-1] - 1 < min_z_tuple:
             e = (n,(n[0],n[1] - 1,n[-1] - 1))
             add_edge_arr.append(e)
    
    
    for e in add_edge_arr:
        G.add_edge(*e)
        

    return G
G = get_3d_graph()

In [132]:
def tupel_to_tupel_arr(tupel):
    x,y,z = tupel
    fl = 20*z + 100
    match_z_arr = find_z_arr(fl)

    found = False 
    for cnt, tupel_set in enumerate(climate_df.loc[climate_df["FL"] == match_z_arr,"tupel"].to_numpy()):
        for cnt_2, test_tuple in enumerate(tupel_set):
            if tupel == test_tuple:
                
                found = True 
                break
        if found:
            break
    
    return tupel_set, cnt_2
tupel_to_tupel_arr((29,5,11))

(((29, 5, 10), (29, 5, 11), (nan, nan, nan)), 1)

In [303]:
# for path in tupel_path_arr:
def tupel_path_to_trajec(tupel_path, start_index = 0, start_time=""):
    shifted_path = []
    for tupel in tupel_path:
        tupel_arr, t_inx = tupel_to_tupel_arr(tupel)
        x = climate_df.loc[climate_df["tupel"] == tupel_arr,"LONGITUDE"].to_numpy()[0]
        y = climate_df.loc[climate_df["tupel"] == tupel_arr,"LATITUDE"].to_numpy()[0]
        z = climate_df.loc[climate_df["tupel"] == tupel_arr,"FL"].to_numpy()[0][t_inx]

        shifted_path.append({"x":x,
                             "y":y,
                             "z":z
                            })

    if start_time == "":
        shifted_path[0]["t"] =  np.datetime64('2018-06-23 '+ flight_df["start_time"].iloc[start_index]) 
        shifted_path[0]["delta_t"] = np.timedelta64(0,'s')
    else:
        shifted_path[0]["t"] =  start_time[0]
        shifted_path[0]["delta_t"] = start_time[1]

    for i in range(len(shifted_path)-1):
        
        point_x = x_to_km(np.abs(shifted_path[i+1]["x"] - shifted_path[i]["x"]) )
        point_y = y_to_km(np.abs(shifted_path[i+1]["y"] - shifted_path[i]["y"]) )
        point_z = shifted_path[i+1]["z"] - shifted_path[i]["z"]

        if point_z == 0:
            point_dist = point_x + point_y
            point_t = int(3600 * point_dist/ (find_fuel(cruise_df,shifted_path[i+1]["z"])[0] * 1.852))
        
        elif point_z > 0:
            z_0 = shifted_path[i]["z"]
            z_1 = shifted_path[i+1]["z"]
            
            v_0, roc_0, _ = find_fuel_climb(climb_df, z_0)
            v_1, roc_1, _ = find_fuel_climb(climb_df, z_1)
            delta_t_c_0 = 60*1000/(roc_0) # in seconds 
            delta_t_c_1 = 60*1000/(roc_1) # in sec 
            
            dist_climb = np.sqrt((delta_t_c_0*v_0* 1.852/3600 + delta_t_c_1*v_1* 1.852/3600)**2 - 0.61**2) # km
            
            point_dist = point_x + point_y - dist_climb
            point_t = int(3600 * point_dist/ (find_fuel(cruise_df,shifted_path[i+1]["z"])[0] * 1.852)) 
            point_t += int(delta_t_c_0 + delta_t_c_1)
             
        elif point_z < 0:
            z_0 = shifted_path[i]["z"]
            z_1 = shifted_path[i+1]["z"]
            
            v_0, roc_0, _ = find_fuel_climb(descent_df, z_0)
            v_1, roc_1, _ = find_fuel_climb(descent_df, z_1)
            delta_t_c_0 = 60*1000/(roc_0) # in seconds 
            delta_t_c_1 = 60*1000/(roc_1) # in sec 
            
            dist_climb = np.sqrt((delta_t_c_0*v_0* 1.852/3600 + delta_t_c_1*v_1* 1.852/3600)**2 - 0.61**2) # km
            
            point_dist = point_x + point_y - dist_climb
            point_t = int(3600 * point_dist/ (find_fuel(cruise_df,shifted_path[i+1]["z"])[0] * 1.852)) 
            point_t += int(delta_t_c_0 + delta_t_c_1)
        else: 
            print("Here, sth wrong", point_x, point_y, point_z)
            

        shifted_path[i+1]["t"] = shifted_path[i]["t"] + np.timedelta64(point_t,'s')
        shifted_path[i+1]["delta_t"] = np.timedelta64(point_t,'s')
    
    return shifted_path
# tupe'l_path_to_trajec([(1,1,1),(1,0,0)]),tupel_path_to_trajec([(1,1,1),(1,0,1)]) 

In [134]:
m_val = -np.min(climate_df["MERGED"].to_numpy())
def c_weight(df, tupel_0, tupel_1):
    
    tupel_arr_0, t0_pos = tupel_to_tupel_arr(tupel_0)
    tupel_arr_1, t1_pos = tupel_to_tupel_arr(tupel_1)
     
    m_0 = df.loc[df["tupel"] == tupel_arr_0,"MERGED"].mean() + m_val# take mean of merged for 3 times 
    m_1 = df.loc[df["tupel"] == tupel_arr_1,"MERGED"].mean() + m_val
    
    
    fl_0 = df.loc[df["tupel"] == tupel_arr_0,"FL"].to_numpy()[0][t0_pos] 
    fl_1 = df.loc[df["tupel"] == tupel_arr_1,"FL"].to_numpy()[0][t1_pos]
    
    trajec = tupel_path_to_trajec([tupel_0,tupel_1])
    c_fuel, _ = consumed_fuel(trajec)
    
    weight = (m_0+m_1)*c_fuel/2
    
    return weight  # kg , scaled by *10**(12)

c_weight(climate_df,(0, 12, 12), (1, 12, 12)  )

19.990095204117797

In [135]:
# edge weights have to be mult by fuel cons already 

def add_weights_graph(G):
    
    for e in G.edges():
        try:
            G[e[0]][e[1]]["w"] = c_weight(climate_df, e[0], e[1])
        except:
            1
    return 0 

In [136]:
# pick either 128 256 512


def gen_rand_path(x_s, y_s, z, x_e, y_e, size):
    rand_paths = [itertools.islice(nx.shortest_simple_paths(G, source= xyz_to_tupel( x_s, y_s, z), target=xyz_to_tupel( x_e, y_e, z_e), weight="w"), size) for z_e in z_arr ]

    tupel_path_arr = []
    try:
        for path_z in list(rand_paths):

            for path in path_z:
                tupel_path_arr.append(path)
            #         print(nx.path_weight(G,path,"w"))
    except:
        1
    return tupel_path_arr


In [137]:
def convertTuple(tup):
    st = ''.join(map(str, tup))
    return st

def grover_search(n_qubits, index):
    n_iter = int(np.pi/4 * np.sqrt(2**n_qubits))
    aer_simulator = Aer.get_backend('aer_simulator')
    lst = list(itertools.product([0, 1], repeat=n_qubits))
    sv_labels = [convertTuple(l) for l in lst]
    sv_label = sv_labels[index]
    
    oracle = Statevector.from_label(sv_label)
    problem = AmplificationProblem(oracle, is_good_state=[sv_label])

    grover = Grover(quantum_instance=aer_simulator, iterations=n_iter)
    result = grover.amplify(problem)
   
    
    res = result.top_measurement if result.oracle_evaluation else False
    found_inx = np.where(np.array(sv_label) == res)[0][0]
    
    return str(res), found_inx
#grover_search(N=5, index=10, n_iter=10)

In [36]:
# q_optimized_traj_arr = []
# for s_inx, (_, t, z, x_s, y_s, x_e, y_e) in enumerate(flight_df.to_numpy()):
#     test_trajec = [tupel_path_to_trajec(tupel_path, start_index=s_inx) for tupel_path in gen_rand_path(x_s, y_s, z, x_e, y_e, size = 2)]
#     for _ in range(32-len(test_trajec)):
#         test_trajec.append(test_trajec[0])
#     q_optimized_traj_arr.append(test_trajec)

In [37]:
# np.save("../data/q_optimized_traj_arr.npy",np.array(q_optimized_traj_arr,dtype=object))

In [38]:
q_optimized_traj_arr = np.load("../data/q_optimized_traj_arr.npy", allow_pickle=True)

# Run on hardware

In [105]:
from qiskit.tools.monitor import job_monitor

aer_simulator = Aer.get_backend('aer_simulator')

N=5
index = 1
n_iter = 4
lst = list(itertools.product([0, 1], repeat=N))
sv_labels = [convertTuple(l) for l in lst]
sv_label = sv_labels[index]

oracle = Statevector.from_label(sv_label)
problem = AmplificationProblem(oracle, is_good_state=[sv_label])

grover = Grover(iterations=4)


transpiled_grover_circuit = transpile(grover.construct_circuit(problem), device, optimization_level=3)
job = device.run(transpiled_grover_circuit)
job_monitor(job, interval=2)

# result = grover.amplify(problem)

Job Status: job has successfully run


In [106]:
results = job.result()
results
# answer = results.get_counts(grover.construct_circuit(problem))
# plot_histogram(answer)

Result(backend_name='ibmq_belem', backend_version='1.0.41', qobj_id='cd78332f-c6f1-41d6-ab14-503c071b9036', job_id='625e5dde19e670c1bf1f7978', success=True, results=[ExperimentResult(shots=4000, success=True, meas_level=2, data=ExperimentResultData(counts={}), header=QobjExperimentHeader(qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4]], n_qubits=5, qreg_sizes=[['q', 5]], clbit_labels=[], memory_slots=0, creg_sizes=[], name='Grover circuit', global_phase=2.356194490192344, metadata={}))], date=2022-04-19 08:59:52+02:00, status=Successful completion, status=QobjHeader(backend_name='ibmq_belem', backend_version='1.0.41'), execution_id='4c0d4a96-bfae-11ec-959c-b02628f7f59e', time_taken=5.955143451690674, error=None, client_version={'qiskit': '0.32.1'})

In [ ]:
def grover_search_hw(n_qubits, index):
    n_iter = int(np.pi/4 *np.sqrt(2**n_qubits))
    lst = list(itertools.product([0, 1], repeat=n_qubits))
    sv_labels = [convertTuple(l) for l in lst]
    sv_label = sv_labels[index]
    
    oracle = Statevector.from_label(sv_label)
    problem = AmplificationProblem(oracle, is_good_state=[sv_label])

    grover = Grover(quantum_instance=device, iterations=n_iter)
    
    result = grover.amplify(problem)
    
    res = result.top_measurement 
    found_inx = np.where(np.array(sv_label) == res)[0][0]
    
    return str(res), found_inx

grover_search_hw(5, 15)

In [ ]:
findMinCostQuantum(state_arr, basis_state_to_trajec,True)

In [73]:
IBMQ.save_account('661471246b9d4562e2482696501b3334b2b2dc7f215ff96a3427581b1808dd04666ad023f0bd5c119028d979903872515e1d50fbe277a26aaaace1ac9961ac8f')

In [75]:
provider = IBMQ.load_account()
provider = IBMQ.get_provider("ibm-q")
device = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 3 and 
                                   not x.configuration().simulator and x.status().operational==True))
print("Running on current least busy device: ", device)

ibmqfactory.load_account:WARNING:2022-04-19 08:15:13,371: Credentials are already in use. The existing account in the session will be replaced.


Running on current least busy device:  ibmq_belem


In [ ]:
from qiskit.tools.monitor import job_monitor
transpiled_grover_circuit = transpile(grover_circuit, device, optimization_level=3)
# job = device.run(transpiled_grover_circuit)
# job_monitor(job, interval=2)

In [ ]:
results = job.result()
answer = results.get_counts(grover_circuit)
plot_histogram(answer)

In [68]:
# np.save("../data/res_q_sim.npy",np.array(res_q_sim,dtype=object))

# De-Conflicting

List of trajecs with overlap 
each qubit is a decision if point of conflict is resolved by +-20 FL 
The structure needed for that is the traj, pos of conflict in the trajectory and the list of all trajecs

Resolving is done by looking at direc of travel and the introdocung either (x+-1,y,z+-1) or (x,y+-1,z+-1)



In [238]:
# generate list with points of conflict 

def constraint_n_planes(trajec_arr):
    flat_traj = []
    for cnt_1, sublist in enumerate(trajec_arr):
        for cnt_2, item in enumerate(sublist):
            temp_list = list(item.values())
            temp_list.append(cnt_1)
            temp_list.append(cnt_2)
            flat_traj.append(temp_list) # trajec index and point in trajec index
    flat_traj = np.array(flat_traj)


    _, index, count = np.unique(np.array(flat_traj[:,0:3], dtype=float),axis=0,  return_index=True,return_counts=True)

    # remove if xyz are only in complete traj once 
    rem_index = []
    
    for (i,c) in zip(index, count):
        if c == 1:
            rem_index.append(i)
            

    rem_flat_traj = np.delete(flat_traj, rem_index,axis=0)

    # if time difference shows that they are in voxel at the same time 
    overlap_dict = {}
    for cnt, check_arr in enumerate(rem_flat_traj):
        for test_arr in rem_flat_traj[cnt+1:]:
            if  np.array_equal(check_arr[0:3], test_arr[0:3]): 
                time_diff = np.abs(check_arr[3] - test_arr[3])
                if time_diff < check_arr[4]:
                    
                    temp_pos = (check_arr[-2],check_arr[-1])
                    if temp_pos in overlap_dict:
                        overlap_dict[temp_pos] += [test_arr]
                    else:
                        overlap_dict[temp_pos] = [test_arr]

    cnt_confl = 0 
    important_confl_dict = {}
    for k, v in overlap_dict.items():
        if len(v)>2:
            cnt_confl += len(v)
            important_confl_dict[k] = v
    
    return cnt_confl, important_confl_dict
c, overlap_dict = constraint_n_planes(res_q_sim)

In [215]:
def xyz_to_tupel(x, y, z):
    if math.isnan(z):
        return (float("nan"),float("nan"),float("nan"))
    else:
        return (  int((x+30)/2), int((y-34)/2), int((z-100)/20))

In [408]:
from copy import deepcopy

In [416]:
# take binary value and and translate it to changed traj  arr
def binary_to_traj(b_val, conflict_point, input_traj):
    
    traj = deepcopy(input_traj)
    
    con_p_m1 = traj[conflict_point[0]][conflict_point[1]-1]
    m_tup = xyz_to_tupel(con_p_m1["x"], con_p_m1["y"], con_p_m1["z"])

    con_p = traj[conflict_point[0]][conflict_point[1]]
    
    # problem dont allow to go beyond boundaries 
    # sol remove 
    
    con_p["z"] += (-20 + b_val*40)
    change_tup = xyz_to_tupel(con_p["x"], con_p["y"], con_p["z"])
    

    con_p_p1 = traj[conflict_point[0]][conflict_point[1]+1]
    p_tup = xyz_to_tupel(con_p_p1["x"], con_p_p1["y"], con_p_p1["z"])
    
    
    temp_path = tupel_path_to_trajec([m_tup, change_tup], start_time = [con_p_m1["t"],con_p_m1["delta_t"]])
    new_t = temp_path[1]["t"]
    new_delta_t = temp_path[1]["delta_t"]

    con_p["t"] = new_t
    con_p["delta_t"] = new_delta_t

    temp_path = tupel_path_to_trajec([change_tup, p_tup], start_time = [con_p["t"],con_p["delta_t"]])
    new_t = temp_path[1]["t"]
    new_delta_t = temp_path[1]["delta_t"]

    con_p_p1["t"] = new_t
    con_p_p1["delta_t"] = new_delta_t
    
    return traj 

In [412]:
def bitstr_to_traj(bit_string, confl_arr, input_traj):
    traj = deepcopy(input_traj)
    for bit, confl in zip(bit_string, confl_arr):
        changed_traj = binary_to_traj(bit, confl, traj)
    return changed_traj


In [453]:
# iterate through bitstring and alter all trajecs 
bit_string = (0,1,0,1,0)
confl_list = np.array(list(overlap_dict.keys()))
choice_inx = np.random.choice(np.arange(len(confl_list)),size=5)
confl_arr = confl_list[choice_inx]
confl_arr

array([[43, 33],
       [58, 22],
       [17,  6],
       [11, 29],
       [ 3, 32]])

In [343]:
flight_df["cost"] = [C(traj) for traj in res_q_sim]
flight_df

,flight_number,start_time,start_flightlevel,start_longitudinal,start_latitudinal,end_longitudinal,end_latitudinal,cost
0,0,07:15:00,250,-30,56,24,60,3.290849e-08
1,1,07:10:00,310,-30,56,30,42,5.423837e-08
2,2,07:50:00,250,-22,34,30,46,4.770779e-08
3,3,07:30:00,260,-26,34,28,60,8.359474e-08
4,4,07:00:00,230,-30,42,30,46,4.596743e-08
...,...,...,...,...,...,...,...,...
94,95,06:10:00,260,-30,40,26,60,5.849638e-08
95,96,07:05:00,200,-30,42,22,60,5.734088e-08
96,97,06:00:00,300,-14,34,30,52,4.471082e-08
97,98,07:30:00,310,-30,40,30,56,6.344482e-08


In [438]:
def set_new_cost(confl_arr, de_conflicted_traj_arr):
    for row_inx in confl_arr[:,0]:
        flight_df.iat[row_inx, flight_df.columns.get_loc('cost')] = C(de_conflicted_traj_arr[row_inx])
    return np.sum(flight_df["cost"].to_numpy())

def get_new_cost(confl_arr, de_conflicted_traj_arr):
    c_arr = flight_df["cost"].to_numpy()
    c_sum = 0 
    for cnt,c in enumerate(c_arr):
        if cnt not in confl_arr[:,0]: 
            c_sum += c
    for c_inx in confl_arr[:,0]:
        c_sum += C(de_conflicted_traj_arr[c_inx])
    return c_sum

In [436]:
res_q_sim = np.load("../data/res_q_sim.npy",allow_pickle=True)[:,1]

In [455]:
def vqe_cost(bit_string, confl_arr, traj_arr):
    traj_arr = traj_arr[:]
#     initial_cost = np.sum([C(traj) for traj in traj_arr])
    
    p = 0.1
#     inital_penalty = p*constraint_n_planes(traj_arr)[0]
#     initial_cost = np.sum([C(traj) for traj in traj_arr])
    
    de_conflicted_traj_arr = bitstr_to_traj(bit_string, confl_arr, traj_arr )
    
    improved_cost = get_new_cost(confl_arr, de_conflicted_traj_arr)
    penalty = p*constraint_n_planes(de_conflicted_traj_arr)[0]
    
#     print(initial_cost, inital_penalty)
#     print(improved_cost, penalty/   5e7)
    
    return improved_cost + penalty/5e7
vqe_cost(bit_string, confl_arr, res_q_sim)

8.268680971748977e-06

In [403]:
import math
import numpy as np
import matplotlib.pyplot as plt
from pytket.circuit import Circuit
from pytket.extensions.qiskit import AerStateBackend, AerBackend  
from pytket.circuit.display import render_circuit_jupyter
from pytket.utils.results import probs_from_state  
from qiskit.quantum_info import partial_trace, entropy, Statevector  


In [255]:
def ansatz(params):
    c = Circuit(n_qubits,0)
    params = params / np.pi  # pytket works in units of pi: Ry(p, q) = exp(-i pi p/2 Y_q)    c = Circuit(n_qubits)
    param_index = 0
    for q in range(n_qubits):
        c.Ry(params[param_index], q)
        param_index += 1
    for l in range(n_layers):
        for q in range(n_qubits-1):
            c.CX(q, q+1)
        for q in range(n_qubits):
            c.Ry(params[param_index], q)
            param_index += 1
    return c

n_qubits = 5
n_layers = 1  # number of ansatz circuit layers
n_params = n_qubits * (1 + n_layers)  


In [256]:
render_circuit_jupyter(ansatz(np.ones(n_params) * np.pi))

In [290]:
# Filter: function f: c real to f(c) real such that f^2(c) strictly decreases

t = 1  # t>0 is a function parameter. Could be changed at each optimization step

def f(c, t):
    return np.exp(-t * c)

In [291]:
# Sample ansatz circuit with tKet

# number of measurement shots. Each shot is one sample of the bitstring distribution produced by the quantum ansatz
n_shots = 1
backend = AerBackend()

def sample_ansatz(params):
    c = ansatz(params)
    c.measure_all()
    backend.compile_circuit(c)
    handle = backend.process_circuit(c, n_shots=n_shots)
    result = backend.get_result(handle)
    counts = result.get_counts()
    probs = {bitstring: c / n_shots for bitstring, c in counts.items()}  # samples from the bitstring distribution
    return probs

In [299]:
# Evaluate filter expectation value and build up the spectrum

spectrum = {}  # will contain a pair (bitstring, C(bitstring)) for each different sampled bitstring during experiment

def evaluate(probs, t, spectrum):
    filter_exp = 0
    for bitstring, p in probs.items():
        if bitstring not in spectrum:
            cost = vqe_cost(bitstring, confl_arr, res_q_sim)
            spectrum[bitstring] = cost
        else:
            cost = spectrum[bitstring]
        filter_exp += f(cost, t) * p
    return filter_exp, spectrum

In [293]:
# Gradient evaluated at params

def g(params, t, spectrum):
    output = np.zeros(n_params)
    for i in range(n_params):
        params_plus = params + np.bincount([i], None, n_params) * np.pi / 2
        params_minus = params - np.bincount([i], None, n_params) * np.pi / 2
        probs_plus = sample_ansatz(params_plus)
        probs_minus = sample_ansatz(params_minus)
        filter_exp_plus, spectrum = evaluate(probs_plus, t, spectrum)
        filter_exp_minus, spectrum = evaluate(probs_minus, t, spectrum)
        output[i] = filter_exp_minus - filter_exp_plus
    return output, spectrum

In [294]:
# Gradient-based optimiser

learning_rate = 0.01

def update_params(params, gradient):
    return params - learning_rate * gradient

In [445]:
# F-VQE algorithm
    
# initialises the quantum computer in the equal superposition

def fvqe(params, n_steps):
    spectrum = {}
    for step in range(n_steps):
        gradient, spectrum = g(params, t, spectrum)
        params = update_params(params, gradient)
        best_cost = min(spectrum.values())
        best_solutions = {bitstring for bitstring, cost in spectrum.items() if cost==best_cost}
        print(f"step {step}: best cost = {best_cost} for {best_solutions}")
    return spectrum

In [456]:
res_q_sim = np.load("../data/res_q_sim.npy",allow_pickle=True)[:,1]
params = np.array(list(np.zeros(n_params - n_qubits)) + list(np.ones(n_qubits) * np.pi / 2)) 

fvqe(params, n_steps = 3)

step 0: best cost = 8.26854654453518e-06 for {(0, 1, 0, 0, 1), (0, 0, 1, 0, 1), (0, 0, 1, 1, 1), (0, 1, 1, 1, 1)}
step 1: best cost = 8.26854654453518e-06 for {(0, 0, 1, 0, 1), (0, 0, 1, 1, 1), (0, 1, 0, 0, 1), (0, 0, 0, 0, 1), (1, 0, 0, 1, 1), (0, 0, 0, 1, 1), (0, 1, 1, 1, 1), (1, 1, 1, 0, 1), (1, 0, 1, 0, 1), (1, 1, 0, 0, 1)}
step 2: best cost = 8.26854654453518e-06 for {(0, 0, 1, 0, 1), (0, 1, 0, 1, 1), (0, 0, 1, 1, 1), (0, 1, 0, 0, 1), (0, 0, 0, 0, 1), (1, 0, 0, 1, 1), (0, 0, 0, 1, 1), (0, 1, 1, 1, 1), (1, 1, 1, 0, 1), (1, 0, 1, 0, 1), (1, 1, 0, 0, 1)}


{(1, 1, 0, 1, 0): 8.268680971748977e-06,
 (0, 0, 1, 1, 0): 8.268680971748977e-06,
 (1, 1, 1, 0, 0): 8.268680971748977e-06,
 (0, 0, 1, 0, 0): 8.268680971748977e-06,
 (0, 1, 1, 0, 0): 8.268680971748977e-06,
 (0, 0, 0, 0, 0): 8.268680971748977e-06,
 (1, 1, 1, 1, 0): 8.268680971748977e-06,
 (0, 0, 1, 0, 1): 8.26854654453518e-06,
 (1, 0, 1, 0, 0): 8.268680971748977e-06,
 (0, 1, 1, 1, 1): 8.26854654453518e-06,
 (0, 1, 0, 1, 0): 8.268680971748977e-06,
 (0, 0, 1, 1, 1): 8.26854654453518e-06,
 (1, 0, 0, 1, 0): 8.268680971748977e-06,
 (0, 1, 0, 0, 1): 8.26854654453518e-06,
 (0, 1, 0, 0, 0): 8.268680971748977e-06,
 (0, 0, 0, 0, 1): 8.26854654453518e-06,
 (1, 0, 0, 1, 1): 8.26854654453518e-06,
 (1, 1, 1, 0, 1): 8.26854654453518e-06,
 (1, 0, 1, 0, 1): 8.26854654453518e-06,
 (1, 1, 0, 0, 1): 8.26854654453518e-06,
 (0, 0, 0, 1, 1): 8.26854654453518e-06,
 (1, 0, 0, 0, 0): 8.268680971748977e-06,
 (0, 1, 0, 1, 1): 8.26854654453518e-06,
 (0, 1, 1, 1, 0): 8.268680971748977e-06,
 (0, 0, 0, 1, 0): 8.2686809

In [272]:

t = 1  # initial value of tau.
r = 0.1  # threshold for tau scheduling.
learning_rate = 1
n_steps = 5 # number of F-VQE iterations

spectrum = dict()

n_qubits = 5
n_layers = 1  # number of ansatz circuit layers
n_params = n_qubits * (1 + n_layers)

params = np.random.rand(n_params)*np.pi/2
evolution, final_param, spectrum = fvqe(params, t, n_steps,spectrum,cost_function = vqe_cost, allowPrint=True)
plot_evolution(evolution)

AttributeError: 'AerStateBackend' object has no attribute 'get_state'